In [5]:
from Bio import SeqIO
import os
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk


def read_pssm_file(file_path):
    pssm = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[3:]:
            line_list = line.split(' ')
            line_list = [item for item in line_list if len(item)!=0]
            if len(line_list[2:22]) == 0:
                break
            pssm.extend(list(map(int,line_list[2:22])))
    return pssm

def getFasta(fasta_path):
    fasta_file = fasta_path

    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append({
            "id": record.id,
            "sequence": str(record.seq)
        })

    result = []

    for sequence in sequences:

        
        if sequence["sequence"].find('O') != -1:
            print(fasta_path)
            continue
        if len(sequence["sequence"])>512:
            continue

        result.append(sequence["sequence"])


    return result




def getPssm(dim):
    folder_path = r"./PSSMs"

    files = os.listdir(folder_path)
    pssm_labels = []
    pssm_features = []

    temp_label = -1
    queDict = dict()
    for file in files:
        sub_files = os.listdir(folder_path + '/' + file)
        if file == 'NMBP':
            continue
        else:
            temp_label += 1
        queDict[file] = []
        print(temp_label,file)
        for sub_file in sub_files:
            end_file_path = folder_path + '/' + file + '/' + sub_file
            temp = read_pssm_file(end_file_path)
            if len(temp) > 10240:
                continue
            pssm_features.append(temp)
            pssm_labels.append(temp_label)
            queDict[file].append(sub_file.split('.')[0])
    
    new_features = []
    for i in range(len(pssm_features)):
        pssm_features[i].extend([0 for j in range(10240-len(pssm_features[i]))])
        new_features.append(pssm_features[i])

    n_components =dim

    pca = PCA(n_components=n_components)

    X_pca = pca.fit_transform(new_features)
    
    return X_pca,queDict



def getData(que_dict):
    folder_path = r'./Reads'

    label = []
    feature = []

    folders = os.listdir(folder_path)
    cnt = 0
    flag = -1
    for folder in folders:
        filds = os.listdir(folder_path + '/' + folder)
        if folder == 'NMBP':
            continue
        else:
            flag += 1
        print(flag,folder)
        for fild in filds:
            if fild.split('.')[0] not in que_dict[folder]:

                continue
            false_feature = getFasta(folder_path + '/' + folder + '/' + fild)
            
            false_feature = ' '.join(false_feature[0])
            feature.append(false_feature)
            label.append(flag)
    
    return feature,label
    
    
import os
from sklearn.decomposition import PCA
import numpy as np
import torch
def mix_data(encoded_data,X_pca):
    X_pca = cp.deepcopy(X_pca)
    input_ids = []
    attention_mask = []
    for i in range(len(X_pca)):
        list_encoded = encoded_data['input_ids'][i].tolist()
        list_X_pca = list(X_pca[i])
        list_X_pca.extend(list_encoded)
        input_ids.append(list_X_pca)
        attention_mask_list = encoded_data['attention_mask'][i].tolist()
        attention_mask_listtemp = [1 for i in range(1024-514)]
        attention_mask_listtemp.extend(attention_mask_list)
        attention_mask.append(attention_mask_listtemp)
    
    return input_ids,attention_mask

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy as cp
X_pca,queDict = getPssm(512)
featrue,label = getData(queDict)

0 Ca
1 Cu
2 Fe
3 K
4 Mn
5 Zn


In [ ]:
before_featrue = []
for i in featrue:
    before_featrue.append(i.split(' '))

In [ ]:
model = Word2Vec(before_featrue, vector_size=50)

word_vectors = model.wv

vector = word_vectors['A']

In [ ]:
after_feture = []
after_label = []
remove_index = []
maxmax = -1
for i in range(len(before_featrue)):
    temp = []
    continue_flag = 0
    for j in range(len(before_featrue[i])):
        try:
            temp.extend(word_vectors[before_featrue[i][j]])
        except:
            remove_index.append(i)
            continue_flag = 1
            break
    if continue_flag == 1:
        continue
    temp.extend([0 for item in range(25600 - len(temp))])
    after_feture.append(temp)
    after_label.append(label[i])

In [ ]:
after_X_pca = []
for i in range(len(X_pca)):
    if i in remove_index:
        continue
    else:
        after_X_pca.append(X_pca[i])

In [ ]:
pca = PCA(n_components=512)

after_feture_pca = pca.fit_transform(after_feture)

In [ ]:
after_feture_pca = after_feture_pca.tolist()

In [ ]:
all_total_feture = []
for i in range(len(after_X_pca)):
    after_feture_pca_copy = cp.deepcopy(after_feture_pca[i])
    after_feture_pca_copy.extend(after_X_pca[i])
    all_total_feture.append(after_feture_pca_copy)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

all_total_feture_normalized = scaler.fit_transform(all_total_feture)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_total_feture_normalized, after_label, test_size=0.1, random_state=42)

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model_svm = svm.SVC(kernel='linear')  

model_svm.fit(X_train, y_train)

y_pred_svm = model_svm.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("ACC:", accuracy_svm)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


clf_DecisionTreeClassifier = DecisionTreeClassifier(random_state=42)

clf_DecisionTreeClassifier.fit(X_train, y_train)

y_pred_DecisionTreeClassifier = clf_DecisionTreeClassifier.predict(X_test)

accuracy_DecisionTreeClassifier = accuracy_score(y_test, y_pred_DecisionTreeClassifier)
print("ACC：", accuracy_DecisionTreeClassifier)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model_RandomForestClassifier = RandomForestClassifier(n_estimators=100, random_state=42)

model_RandomForestClassifier.fit(X_train, y_train)

y_pred_RandomForestClassifier = model_RandomForestClassifier.predict(X_test)

accuracy_RandomForestClassifier = accuracy_score(y_test, y_pred_RandomForestClassifier)
print("ACC：", accuracy_RandomForestClassifier)


In [ ]:
from catboost import CatBoostClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model_catboost = CatBoostClassifier(iterations=5000, depth=8, learning_rate=0.09)

model_catboost.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

y_pred_prob_catboost = model_catboost.predict_proba(X_test)
y_pred_catboost = np.argmax(y_pred_prob_catboost, axis=1) 

accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print(f'Accuracy: {accuracy_catboost}')


In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',  
    'num_class': 6,  
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,
    'feature_fraction': 0.9
}


num_round = 2500
bst_lightgbm = lgb.train(params, train_data, num_round, valid_sets=[test_data],verbose_eval=100)

y_pred_prob_lightgbm = bst_lightgbm.predict(X_test, num_iteration=bst_lightgbm.best_iteration) 
y_pred_lightgbm = np.argmax(y_pred_prob_lightgbm, axis=1)  


accuracy_lightgbm = accuracy_score(y_test, y_pred_lightgbm)
print(f'Accuracy: {accuracy_lightgbm}')


In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'multi:softmax',
    'num_class': 6, 
    'learning_rate': 0.01
}

watchlist = [(dtrain, 'train'), (dtest, 'test')]

num_round = 4500
bst_xgboost = xgb.train(params, dtrain, num_round, evals=watchlist, verbose_eval=100)


dtest = xgb.DMatrix(X_test, label=y_test)


y_pred_prob_xgboost = bst_xgboost.predict(dtest, output_margin=True) 
y_pred_xgboost = np.argmax(y_pred_prob_xgboost, axis=1)  

accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
print(f'Accuracy: {accuracy_xgboost}')


In [ ]:
class_feture_train = []
y_pred_prob_xgboost = y_pred_prob_xgboost.tolist()
y_pred_prob_lightgbm = y_pred_prob_lightgbm.tolist()
y_pred_prob_catboost = y_pred_prob_catboost.tolist()
y_class_feture_train = cp.deepcopy(y_test)
for i in range(len(y_pred_prob_xgboost)):
    class_feture_train.append(y_pred_prob_xgboost[i] + y_pred_prob_lightgbm[i] + y_pred_prob_catboost[i])


In [ ]:
# class_feture_train

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train_vote, X_test_vote, y_train_vote, y_test_vote = train_test_split(class_feture_train, y_class_feture_train, test_size=0.3, random_state=42)

clf_vote = svm.SVC()  

clf_vote.fit(X_train_vote, y_train_vote)

y_pred_vote = clf_vote.predict(X_test_vote)

accuracy_vote = accuracy_score(y_test_vote, y_pred_vote)
print("ACC:", accuracy_vote)

In [ ]:

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


conf_matrix = confusion_matrix(y_test_vote, y_pred_vote)

classes = ['Ca','Cu','Fe','K','Mn','Zn']
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes,annot_kws={"size": 14})
plt.xlabel('Predicted Labels', fontsize=15)
plt.ylabel('True Labels', fontsize=15)
plt.title('Confusion Matrix', fontsize=18)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig(r'./images/6-Confusion Matrix.png', dpi=500)  

plt.show()


In [ ]:


from sklearn.metrics import precision_score, recall_score, f1_score


precision_vote = precision_score(y_test_vote, y_pred_vote, average='weighted')
recall_vote = recall_score(y_test_vote, y_pred_vote, average='weighted')
f1_vote = f1_score(y_test_vote, y_pred_vote, average='weighted')
print('mymodel')
print("Precision: {:.4f}".format(precision_vote))
print("Recall: {:.4f}".format(recall_vote))
print("F1 Score: {:.4f}".format(f1_vote))

precision_svm = precision_score(y_test, y_pred_svm, average='weighted')
recall_svm = recall_score(y_test, y_pred_svm, average='weighted')
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')
print('svm')
print("Precision: {:.4f}".format(precision_svm))
print("Recall: {:.4f}".format(recall_svm))
print("F1 Score: {:.4f}".format(f1_svm))

precision_DecisionTreeClassifier = precision_score(y_test, y_pred_DecisionTreeClassifier, average='weighted')
recall_DecisionTreeClassifier = recall_score(y_test, y_pred_DecisionTreeClassifier, average='weighted')
f1_DecisionTreeClassifier = f1_score(y_test, y_pred_DecisionTreeClassifier, average='weighted')
print('DecisionTreeClassifier')
print("Precision: {:.4f}".format(precision_DecisionTreeClassifier))
print("Recall: {:.4f}".format(recall_DecisionTreeClassifier))
print("F1 Score: {:.4f}".format(f1_DecisionTreeClassifier))


precision_RandomForestClassifier = precision_score(y_test, y_pred_RandomForestClassifier, average='weighted')
recall_RandomForestClassifier = recall_score(y_test, y_pred_RandomForestClassifier, average='weighted')
f1_RandomForestClassifier = f1_score(y_test, y_pred_RandomForestClassifier, average='weighted')
print('RandomForestClassifier')
print("Precision: {:.4f}".format(precision_RandomForestClassifier))
print("Recall: {:.4f}".format(recall_RandomForestClassifier))
print("F1 Score: {:.4f}".format(f1_RandomForestClassifier))


precision_xgboost = precision_score(y_test, y_pred_xgboost, average='weighted')
recall_xgboost = recall_score(y_test, y_pred_xgboost, average='weighted')
f1_xgboost = f1_score(y_test, y_pred_xgboost, average='weighted')
print('xgboost')
print("Precision: {:.4f}".format(precision_xgboost))
print("Recall: {:.4f}".format(recall_xgboost))
print("F1 Score: {:.4f}".format(f1_xgboost))

precision_lightgbm = precision_score(y_test, y_pred_lightgbm, average='weighted')
recall_lightgbm = recall_score(y_test, y_pred_lightgbm, average='weighted')
f1_lightgbm = f1_score(y_test, y_pred_lightgbm, average='weighted')
print('lightgbm')
print("Precision: {:.4f}".format(precision_lightgbm))
print("Recall: {:.4f}".format(recall_lightgbm))
print("F1 Score: {:.4f}".format(f1_lightgbm))

precision_catboost = precision_score(y_test, y_pred_catboost, average='weighted')
recall_catboost = recall_score(y_test, y_pred_catboost, average='weighted')
f1_catboost = f1_score(y_test, y_pred_catboost, average='weighted')
print('catboost')
print("Precision: {:.4f}".format(precision_catboost))
print("Recall: {:.4f}".format(recall_catboost))
print("F1 Score: {:.4f}".format(f1_catboost))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

categories = ['XLC-S-MIBP', 'SVM', 'DT', 'RF', 'xgboost', 'lightgbm', 'catboost']
num_categories = len(categories)
values_set1 = [accuracy_vote, accuracy_svm, accuracy_DecisionTreeClassifier, accuracy_RandomForestClassifier, 
               accuracy_xgboost, accuracy_lightgbm, accuracy_catboost]
values_set2 = [precision_vote, precision_svm, precision_DecisionTreeClassifier, precision_RandomForestClassifier, 
               precision_xgboost, precision_lightgbm, precision_catboost]
values_set3 = [recall_vote, recall_svm, recall_DecisionTreeClassifier, recall_RandomForestClassifier, 
               recall_xgboost, recall_lightgbm, recall_catboost]
values_set4 = [f1_vote, f1_svm, f1_DecisionTreeClassifier, f1_RandomForestClassifier, 
               f1_xgboost, f1_lightgbm, f1_catboost]

bar_width = 0.21

r = np.arange(num_categories)
r1 = r - 1.5*bar_width
r2 = r - 0.5*bar_width
r3 = r + 0.5*bar_width
r4 = r + 1.5*bar_width

colors = sns.color_palette("colorblind", 4)

plt.bar(r1, values_set1, color=colors[0], width=bar_width, edgecolor='grey', label='ACC')
plt.bar(r2, values_set2, color=colors[1], width=bar_width, edgecolor='grey', label='P')
plt.bar(r3, values_set3, color=colors[2], width=bar_width, edgecolor='grey', label='R')
plt.bar(r4, values_set4, color=colors[3], width=bar_width, edgecolor='grey', label='F1')

plt.xlabel('Categories', fontsize=15)
plt.ylabel('Values', fontsize=15)

plt.ylim(0.55, 1.0)

plt.xticks(r, categories, fontsize=8)  
plt.title('Model Evaluation', fontsize=18)

plt.legend(loc='upper right',fontsize=12)


plt.savefig(r'./images/6-Model Evaluation.png', dpi=500) 

plt.show()
